In [18]:
import os
import json
import pandas as pd
import numpy as np

In [19]:
df = pd.read_json("./News_Category_Dataset_v2.json", lines=True)
df.to_json('news_data.json',orient = 'records')
df = pd.read_json("./news_data.json")

In [20]:
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [21]:
num_data_points = len(df)
shuffled_df = df.sample(frac=1)
lens = [int(0.8*len(df)),int(0.9*len(df)),(len(df)-1)]
train_data = shuffled_df[:lens[0]]
val_data = shuffled_df[(lens[0]+1):lens[1]]
test_data = shuffled_df[(lens[1]+1):lens[2]]

train_data = shuffled_df[:20600]
val_data = shuffled_df[20601:22700]
test_data = shuffled_df[22701:24701]



train_data.to_json("./splits/train.json",orient = 'records')
val_data.to_json("./splits/val.json",orient = 'records')
test_data.to_json("./splits/test.json",orient = 'records')


In [22]:
for index, row in shuffled_df.iterrows():
    shuffled_df.at[index,"headline"] = row["headline"] + " " + row["short_description"]

In [23]:
train_data = shuffled_df[:lens[0]]
val_data = shuffled_df[(lens[0]+1):lens[1]]
test_data = shuffled_df[(lens[1]+1):lens[2]]

train_data = shuffled_df[:20600]
val_data = shuffled_df[20601:22700]
test_data = shuffled_df[22701:24701]



train_data.to_json("./splits/train.json",orient = 'records')
val_data.to_json("./splits/val.json",orient = 'records')
test_data.to_json("./splits/test.json",orient = 'records')

In [24]:
for x in train_data["headline"]:
    if x is None:
        print("sfsdfs")

In [25]:
list_of_cat = df["category"].to_list()
print(len(list_of_cat))
set_of_cat = set()
for cat in list_of_cat:
    set_of_cat.add(cat)

category_to_id_map = dict()

i = 0
for cat in set_of_cat:
    category_to_id_map[cat] = i
    i = i + 1

with open("./category_to_id.json", 'w') as fp:
    json.dump(category_to_id_map, fp)

200853


In [26]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer("bert-base-uncased-vocab.txt", lowercase=True)

raw_text = ["Hello, what is your name?", "This is very cool", "I do not like you"]

batch = tokenizer.encode_batch(raw_text, add_special_tokens=False)

In [27]:
classes_frequencies = {}
for key in category_to_id_map:
    classes_frequencies[key] = 0

In [28]:
for index, row in shuffled_df.iterrows():
    classes_frequencies[row["category"]] += 1

print(dict(sorted(classes_frequencies.items(), key=lambda item: item[1])))

{'EDUCATION': 1004, 'CULTURE & ARTS': 1030, 'LATINO VOICES': 1129, 'COLLEGE': 1144, 'ENVIRONMENT': 1323, 'ARTS & CULTURE': 1339, 'GOOD NEWS': 1398, 'FIFTY': 1401, 'ARTS': 1509, 'MONEY': 1707, 'TECH': 2082, 'TASTE': 2096, 'WORLD NEWS': 2177, 'SCIENCE': 2178, 'STYLE': 2254, 'RELIGION': 2556, 'WORLDPOST': 2579, 'GREEN': 2622, 'WEIRD NEWS': 2670, 'MEDIA': 2815, 'CRIME': 3405, 'DIVORCE': 3426, 'IMPACT': 3459, 'WOMEN': 3490, 'WEDDINGS': 3651, 'THE WORLDPOST': 3664, 'PARENTS': 3955, 'HOME & LIVING': 4195, 'BLACK VOICES': 4528, 'SPORTS': 4884, 'COMEDY': 5175, 'BUSINESS': 5937, 'FOOD & DRINK': 6226, 'QUEER VOICES': 6314, 'HEALTHY LIVING': 6694, 'PARENTING': 8677, 'STYLE & BEAUTY': 9649, 'TRAVEL': 9887, 'ENTERTAINMENT': 16058, 'WELLNESS': 17827, 'POLITICS': 32739}


In [29]:
scaled_down_df = shuffled_df.loc[df['category'].isin(["POLITICS", "WELLNESS", "ENTERTAINMENT", "TRAVEL", "STYLE & BEAUTY", "PARENTING"])]

In [30]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

for index, row in scaled_down_df.iterrows():
    tokenized_sentence =  str(row["headline"]).split(" ")

    filtered_sentence = [w for w in tokenized_sentence if not w.lower() in stop_words]
  
    filtered_sentence_list = []

    for w in filtered_sentence:
        if w not in stop_words:
            filtered_sentence_list.append(w)
    
    filtered_sentence = ""
    for w in filtered_sentence_list:
        filtered_sentence = filtered_sentence + w + " ";

    filtered_sentence = filtered_sentence[:-1]

    scaled_down_df.at[index, "headline"] = filtered_sentence


In [31]:
scaled_down_df.iloc[0]["headline"]

'Dianne Feinstein Officially Has A Primary Challenger California Democratic state Senate President Kevin de León running unseat Democratic incumbent.'

In [32]:
len(scaled_down_df)

94837

In [33]:
shuffled_scaled_down_df = scaled_down_df.sample(frac=1)

train_data = shuffled_scaled_down_df[:20600]
val_data = shuffled_scaled_down_df[20601:22700]
test_data = shuffled_scaled_down_df[22701:24701]



train_data.to_json("./splits/train.json",orient = 'records')
val_data.to_json("./splits/val.json",orient = 'records')
test_data.to_json("./splits/test.json",orient = 'records')

In [77]:
category_to_id_map = {'PARENTING': 0, 'STYLE & BEAUTY': 1, 'TRAVEL': 2, 'ENTERTAINMENT': 3, 'WELLNESS': 4, 'POLITICS': 5}

with open("./category_to_id.json", 'w') as fp:
    json.dump(category_to_id_map, fp)

In [25]:
batch[0].pad(8)
batch[0].ids
batch[0].attention_mask
len(batch[0])

8